In [ ]:
from dotenv import load_dotenv
import maritalk
import os
load_dotenv()

print(os.environ['MARITALK_API_KEY'])

sabia3 = maritalk.MariTalk(
        key=os.environ['MARITALK_API_KEY'],
        model="sabia-3"
)

In [ ]:
input_file = ""
output_file = ""

In [ ]:
system_msg = 'Você é um analista financeiro com formação em economia que escreve para um público de investidores geral'


def get_user_prompt(company, fr):
    return f'''### Instrução:
Escreva em Português uma análise sobre a Empresa considerando o Fato Relevante e siga o Formato determinado. A análise deve conter TODOS os elementos especificados no seguinte Formato.

### Formato:
<Título>
<Sentença que Resuma a análise>
<Corpo da análise>
<Considerações Finais>

### Empresa:
{company}

### Fato Relevante:
{fr}

### Resposta: '''

In [ ]:
import pandas as pd

df = pd.read_csv(input_file)
gpt4o_rows = df[df["generator_model"] == "gpt4o"].iloc[:-2].copy()
gpt4o_rows["generator_model"] = "sabia3-enriched"

In [ ]:
generated_texts = [None] * len(gpt4o_rows)
print(len(generated_texts))

In [ ]:
for index, row in gpt4o_rows.iterrows():
    try:
        empresa = row['company']
        fr_conteudo = row['enriched_material_fact']

        instruct = get_user_prompt(empresa, fr_conteudo)

        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": instruct}
        ]

        answer = sabia3.generate(
            messages,
            do_sample=True,
            max_tokens=2000,
            temperature=0.3,
            top_p=0.4
        )["answer"]

        generated_texts[index] = answer
    
    except Exception as e:
        print(f"Erro: {e}")
        continue


In [ ]:
gpt4o_rows["generated_text"] = generated_texts

In [ ]:
# concatena e grava
df_final = pd.concat([df, gpt4o_rows], ignore_index=True)
df_final.to_csv(output_file, index=False)
print("Arquivo salvo!")